In [ ]:
import requests
import json
import time

URL = "https://api.miaburton.com/graphql"

HEADERS = {
    "accept": "*/*",
    "content-type": "application/json",
    "country": "ID",
    "currency": "IDR",
    "language": "en",
    "origin": "https://miaburton.com",
    "referer": "https://miaburton.com/en/eyeglasses/women/2",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
    # ⚠️ must be fresh
    "miaburton-req-seed": "1769248448261",
    "miaburton-req-signature": "10d10d79e7af7130f1cd92d9937170d44a7e19fd5677b14f3f1bec93364a49a4",
    "miaburton-req-timestamp": "1769248448261",
}

QUERY = """
query ProductsAndSiblings($input: ProductFilterInput!) {
  productsAndSiblings(input: $input) {
    items {
      sku
    }
    pageInfo {
      pages
      curr
      next
    }
  }
}
"""

def fetch_all_products():
    all_items = []
    page = 1

    while True:
        payload = {
            "operationName": "ProductsAndSiblings",
            "variables": {
                "input": {
                    "filter": {
                        "category": {"eq": "eyeglasses"},
                        "gender": {"in": ["F", "U"]}
                    },
                    "perPage": 24,
                    "page": page,
                    "sort": {
                        "by": "publishingDate",
                        "direction": "DESC"
                    }
                }
            },
            "query": QUERY
        }

        resp = requests.post(URL, headers=HEADERS, json=payload)
        resp.raise_for_status()
        data = resp.json()

        # 🚨 GraphQL error handling
        if "errors" in data:
            print("GraphQL errors:")
            print(json.dumps(data["errors"], indent=2))
            break

        result = data.get("data", {}).get("productsAndSiblings")

        if result is None:
            print("productsAndSiblings is null. Full response:")
            print(json.dumps(data, indent=2))
            break

        items = result["items"]
        page_info = result["pageInfo"]

        print(f"Page {page}/{page_info['pages']} → {len(items)} items")
        print(items)
        all_items.extend(items)

        if not page_info["next"]:
            break

        page += 1
        time.sleep(0.4)

    return all_items


if __name__ == "__main__":
    products = fetch_all_products()
    print(f"\nFetched total: {len(products)} products")


Page 1/115 → 24 items
[{'sku': '00_538228707'}, {'sku': '00_538228705'}, {'sku': '00_538228651'}, {'sku': '00_538228645'}, {'sku': '00_538228627'}, {'sku': '00_538228555'}, {'sku': '00_538228549'}, {'sku': '00_538228662'}, {'sku': '00_538228606'}, {'sku': '00_538228618'}, {'sku': '00_538228640'}, {'sku': '00_538228614'}, {'sku': '00_538228654'}, {'sku': '00_538206665'}, {'sku': '00_538229665'}, {'sku': '00_538184984'}, {'sku': '00_538229538'}, {'sku': '00_538229570'}, {'sku': '00_538210410'}, {'sku': '00_538229504'}, {'sku': '00_538229485'}, {'sku': '00_538210228'}, {'sku': '00_538229507'}, {'sku': '00_538229397'}]
Page 2/115 → 24 items
[{'sku': '00_538229418'}, {'sku': '00_538229534'}, {'sku': '00_538229621'}, {'sku': '00_538229624'}, {'sku': '00_538229369'}, {'sku': '00_538229364'}, {'sku': '00_538229353'}, {'sku': '00_538229350'}, {'sku': '00_538229346'}, {'sku': '00_538229315'}, {'sku': '00_538229278'}, {'sku': '00_538229268'}, {'sku': '00_538229552'}, {'sku': '00_538229611'}, {'sk

In [ ]:
with open("products.json", "w", encoding="utf-8") as f:
    json.dump(products, f, ensure_ascii=False, indent=2)

print(f"\n✅ Done. Total products fetched: {len(products)}")


✅ Done. Total products fetched: 2748


In [ ]:
products[0]

{'sku': '00_538228707'}